In [1]:
import torch 
from torch import nn, optim
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import StandardScaler

In [14]:
# Training features
Training_features = pd.read_csv('../data/datasets/csv_files/Train_features.csv')
# Testing features
Testing_features = pd.read_csv('../data/datasets/csv_files/Test_features.csv')

In [15]:
# Training labels
Training_labels = pd.read_csv('../data/datasets/csv_files/Train_labels.csv')
# Testing labels
Testing_labels = pd.read_csv('../data/datasets/csv_files/Test_labels.csv')

$$ \textbf{Préparation des données :} $$

$$ \text{Traitement des données d'entrainement} $$ 

In [16]:
Training_features.head()

,shot_type,strength,last_event_type,time_since_last_event,distance_from_last_event,home_team_players,away_team_players,distance_to_net,rebound,speed
0,3,1,1,16,79.479557,5,5,33.0,0,4.967472
1,3,1,0,6,108.503456,5,5,47.0,0,18.083909
2,4,1,4,2,35.057096,5,5,48.0,0,17.528548
3,2,1,1,42,31.064449,5,5,60.0,0,0.739630
4,1,1,2,1,13.601471,5,5,37.0,0,13.601471


In [17]:
std = StandardScaler()

In [18]:
training_feature_values = std.fit_transform(Training_features)

In [19]:
# Conversion des données d'entrainement en np.ndarray
training_label_values = Training_labels.values

# Conversion des données d'entrainement en tensor 
X_train = torch.FloatTensor(training_feature_values)
y_train = torch.FloatTensor(training_label_values)

$$ \text{Traitement des données de test} $$ 

In [20]:
# Conversion des données de test en tensor
X_test = torch.FloatTensor(Testing_features.values)
y_test = torch.FloatTensor(Testing_labels.values)

$$\text{Définition de notre réseau de neurones} $$

In [26]:
input_dim = 10
hidden_dim_1 = 100
hidden_dim_2 = 300
hidden_dim_3 = 100
output_dim = 1

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim_1, hidden_dim_2, hidden_dim_3, output_dim):
        super(NeuralNetwork, self).__init__()
        self.layer_1 = nn.Linear(input_dim, hidden_dim_1)
        self.layer_2 = nn.Linear(hidden_dim_1, hidden_dim_2)
        self.layer_3 = nn.Linear(hidden_dim_2, hidden_dim_3)
        self.layer_4 = nn.Linear(hidden_dim_3, output_dim)
       
    def forward(self, x):
        # Choix de la sigmoide car elle est plus utilisée dans les problèmes de classification
        x = torch.nn.functional.relu(self.layer_1(x))
        x = torch.nn.functional.relu(self.layer_2(x))
        x = torch.nn.functional.relu(self.layer_3(x))
        x = torch.nn.functional.sigmoid(self.layer_4(x))
        return x

In [28]:
# Déclaration du modèle
model = NeuralNetwork(input_dim, hidden_dim_1, hidden_dim_2, hidden_dim_3, output_dim)

In [30]:
model.parameters

<bound method Module.parameters of NeuralNetwork(
  (layer_1): Linear(in_features=10, out_features=100, bias=True)
  (layer_2): Linear(in_features=100, out_features=300, bias=True)
  (layer_3): Linear(in_features=300, out_features=100, bias=True)
  (layer_4): Linear(in_features=100, out_features=1, bias=True)
)>

In [31]:
# Set the criterion of the model to measure error
criterion = nn.BCELoss()
# Choose our Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [32]:
num_epochs = 100
loss_values = []


for epoch in range(num_epochs):
    model.train()
    y_pred = model.forward(X_train)
    loss = criterion(y_pred, y_train)
    loss_values.append(loss.detach().numpy())

    if epoch%10 == 0 :
        print(f'Epoch : {epoch} and loss :{loss}')

    # Back propagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print("Training Complete")


Epoch : 0 and loss :0.7151914834976196
Epoch : 10 and loss :0.36117297410964966
Epoch : 20 and loss :0.34099191427230835
Epoch : 30 and loss :0.30278271436691284
Epoch : 40 and loss :0.28951066732406616
Epoch : 50 and loss :0.28561660647392273
Epoch : 60 and loss :0.283051460981369
Epoch : 70 and loss :0.28145402669906616
Epoch : 80 and loss :0.28032615780830383
Epoch : 90 and loss :0.2793436050415039
Training Complete
